In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/tarin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = 'real_poem_jpn.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 22337
total chars: 541


In [3]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 7433


In [4]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [8]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
7433/7433 [==============================] - 12s 2ms/step - loss: 4.7962

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "いとど知らるる
はかりなき千尋の底の海松ぶさの生ひゆくすゑはわれのみぞ見む
千尋"
いとど知らるる
はかりなき千尋の底の海松ぶさの生ひゆくすゑはわれのみぞ見む
千尋ににものむにののののにのにののののにののにののののののののののにのののものののにののののにものにののののののににののにのそものののににののものののののののののむのののののににののののもののものののにののののにもにのもののののににのののののにののにににのののののももののものにものののののにののにのにもののにのにむののものにのののにののにののにのもののにのののののにののののののののにもにののののにのももののにににのにむにのにののののののににのににののににののののもののにのにののむにににににもののにのにのもののののののののにのににものののにものののののにののにのにもののにののにのものののむのののにののにののののにののののににのにのむににのむにのののものののにのににのののののののののののににのののののにののにのもにののもののものもののののにののののにのののののにのににのののののにのにのにのもののに
----- diversity: 0.5
----- Generating with seed: "いとど知らるる
はかりなき千尋の底の海松ぶさの生ひゆくすゑはわれのみぞ見む
千尋"
いとど知らるる
はかりなき千尋の底の海松ぶさの生ひゆくすゑはわれのみぞ見む
千尋のいににれのからの人の松のるににももけの数にもに
にのにととぬてまのへのにしつの世のひのにひにを立の世人の浦もにものはのにのむののにのむををものうのにににのはにまむのむのののやももにののたのりらもの井につとのいのにののののにかやむにはににのれの松もに波にむのしもむなのえにに岩思どの思のにめの行のの野にのにもの道もそも
の薪むにぞもにかもも契むのぎにまいのけもものむもりにむのもものねにもむにゆにのののずにてののぞのど
の中も

うらへるろのによるあをいかつらめけしの松まりありにに朝雨きしきの袖をすれあのぶゆかなしきそべきあぼりを見だかがたちむを
かく寝なかけきけん
大結なかは
つれて言ぞにとどりをけて
は海しのうちを思ちからりをばいかはへよ
のうれてた逢人ばりにだつじべきたちそ葉とよれけしらふとをなしれきにれくもい月だはおゐて見てはかろへてつるけきいかぬは木を
なるらさほぶるゞのし路まべき
つれにかむ匂の世をととくにな

君間ふ
心つき木のことにのぶはらま根す
ならなか旅の
おるすら言うき
露見く空や又ろもろたかきの根とはなりがらむ
たろに谷が大旅
つらむすにか吹らきそに曇じゝ雲まとぞはかき
聞守と舟ことり身ならづらもぬり
露のきとはやなるならならかむ
のかこそにもたらき身ばた秋
----- diversity: 1.2
----- Generating with seed: "とまらぬ露の世をたれか草葉の上とのみ見ん
いにしへの秋の夕の恋しきにいまはと見え"
とまらぬ露の世をたれか草葉の上とのみ見ん
いにしへの秋の夕の恋しきにいまはと見えれもが道めと鈴なしき
たかざわをこそえすかがつむ
山ら姫でらゐの香ばなく知らや野荒の明にとつままやは末な花

友み見りは袖うきへき花かげく君かめきをきをおきけも知ら立きげのうきことにかなき池たどゑなら別身をやつるらはかめけきれかなるる
も見かきをき夢そはがきばの影露に露との竹ださとなり音に心や空
し守のあ人をは中はじばきしよ月焼そへだづどはれき
そすづ雲清ぞまなぐの露に雲ゑのうちならぬゐむ我ぞぬかなや
薄きの雲にたへりはらに霜らせゝ
亡ながらむゆきす寝な
雨にろの
嶺れそにる色をやかがけき心の流づろは身川しにあるらならぬらにばもこそ蛭ほめに行朝罪れれどけらす宿うき
やとどまききより潮はあらをげき治の色をもとと君
下うろきに恋はりれ松ろ
い日人を結ねきひ山の桜ゝろひをき老と生ひ宿ゆりうき
いつはのかへ根をなかへともうら見る
じとべき
あひり蕨らきふ花山ぼ住にとくらしりゞ涙らべきろれき
あらら
Epoch 5/60
7433/7433 [==============================] - 13s 2ms/step - loss: 3.4687

----- Generating text after Epoch: 4
----- divers

なししき
あふれとと思やどしめし
見し人のをとりぬらに
ひとりすれぬるとながめしきてかたしならぬを
なし人のたちとなく
いとならしきしかしきよりににかかしき
あふれと
いととと知れど
----- diversity: 0.5
----- Generating with seed: "長きよの契むすべる草のいほりは
牡鹿なく秋の山ざといかならむ小萩が露のかかる夕ぐ"
長きよの契むすべる草のいほりは
牡鹿なく秋の山ざといかならむ小萩が露のかかる夕ぐとも思ひをくれぬゑととめすれぬみはととなく
いととたにしくけきをととらむすたにしるるかなしき
あふればかはとはすれとなしばしおとりがめにすらぬかな
しほのかたにとりぬらむ
さことはいかなしの袖とたえめづる
いづれとかける心にととられぬる露のととと見れ
あとがためつる
あとめれしきしづととりばかすみにぞ見る
花がたのかげにかへぬらしをと見むかながらしかためてわがめや
いとなきにうへて見しからむことながめや
いとと思はけし
あやととめむすみはぬらしげのかたに心を
かすれてあはれと知らぬ宿におならましせむ
なしりしくて花のたづとにへりしき
うとはらしげく見る
秋のつれとや思ひをむかしき
心とりにしけてながばかせしき
ととりぬらに
つつれどものすましとも恋ふるかながたのうちはたづぬ涙にとぎりしき
花の香のかげはかことにしほれてあとにとややはすみすれと見し人のかたりに
思ふをとぞふるうらなししき
か
----- diversity: 1.0
----- Generating with seed: "長きよの契むすべる草のいほりは
牡鹿なく秋の山ざといかならむ小萩が露のかかる夕ぐ"
長きよの契むすべる草のいほりは
牡鹿なく秋の山ざといかならむ小萩が露のかかる夕ぐもおとゆくと根ぬるる木なしたに
山れはれと見し影はらむ宿とりしけた
いつととを軒ふとなししき
君との露にいづにをうきはれととの衣をやかすみそはつる須とのかとしほののとと唐めどやける月と
がれなきに恋ふればつむとのいととりすれと見したきと吹に
かけれどものかしこの世を
め世ころも知らます月し影べば
明けりれむ
と思ひをしみつむ
見し大ばかこめにすか木は浮もたたそむとの花とながきも露のえぬゝにあふかなは思やどはあすがやもとこそぎる空なならむ
蓮こそ言れぞとと見し
おらし春の露をかけるな

間風のみと見雨よん清つもなども年葉ち縄言がののし色もあきぐ知らず衣たえしろの花亡ちかへいつまし原の色まはさへな
生ぼての春またへし聞くと契らぬ夢にすみひと知らまさひのわかしましせば松絶えてだ霧ろめだのよちに心が寄み風どもし笛の晴やはとぞ見せと藤住たれる旅のばかかへりの小深のさきも影ぞ見ぼ
別れめ吹きへりさへりみけきの桜にひけるぞかけさの語れる峰のねたる心に行をわが身ぞなきかざし世に思ひをあへず結れましふ花の世のへに心はおどりけらしる雁こぼのはたく山な
うは経のわびみ風吹も思だのみづらあはれと寝ならとや
立ちが瀬に小露がをたるまできな
来き世の山にたちはなびとけて絶え
Epoch 12/60
7433/7433 [==============================] - 12s 2ms/step - loss: 1.3119

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "けふはかけじと忍ぶれど心のうちにものぞかなしき
ふりすててけふはゆくとも鈴鹿河八"
けふはかけじと忍ぶれど心のうちにものぞかなしき
ふりすててけふはゆくとも鈴鹿河八十たたれてあはれとやまし
秋のたたにとぞ見む
ひとりにて立ちからまどはねを見しける
もろの涙をみはたづはとけふの露にかける心は知らずそりけふの露にととを思はむ
たるしとりても君にかくやはすらむ
ひとりにてて見しほどぞぬる
いづはとにふるかなりしを
うへてて見て見るこの秋のかはらにをむかならぬにとこならむ
秋のへてかけて見る
時はとのあふとの思ふを知らぬとの露にかことを
うきの小葉のはうははれのあま舟ぞはかけしをらむととも思まねを
うき世のうはみにぞ見ることのよりにを思ふうち
いづとも知らぬをしたどもかくとの思ふかにぞかすべき
雪ふるかは知らぬ世のうきにぞをくれぬをとがむかたにおらるゝけるのかげを見ぬ心ちはたづる雲の露にかからを
うきの世にあめぬるるかな
やれはとけふの山のたびはととを思ふらの色に心はわかにぞとを見ぬかな
めてて世のうきにぞとを見ぬかな
うきはらしに吹らむことならぬとな
うき
----- diversity: 0.5
----- Generating with seed: "けふはかけじ

山がにるあとをおめても知らかげをしけてやづるものさむかな
さめにつもあらぬになるなき人のたに心はりば
いとに消にうらむとをながめや
いと
----- diversity: 1.0
----- Generating with seed: "ぞやも花のさかりにひとめ見し木のもとさへや秋はさびしき
桜こそ思しらすれ咲きにほ"
ぞやも花のさかりにひとめ見し木のもとさへや秋はさびしき
桜こそ思しらすれ咲きにほかあはおどものえぞかはらむことをたづけわがたにしはらがむるばしかたるゝにおめあらずはせむ
いふつあてうためしやまるかよりけてか雪にの見る
身けさはをわかに立つるも露にかへぬさん人なりかなを
ひのまることはかに尋の影はかけ見る野木のうねに心やりせむ
をきがへてかげく見る人に負けるれぬしとめあめず
ち知れずてやれる見まどぞもふかがらし雲をいづねにかめてりは見ぬ
世にことはれる木にぞとためてぞ見ばもかにかはすかな身を
さす過ぐしげむきよへてて世は見る心に
千とを霧る心にと姫にうへる心みそふる木のうみはねてさはもりき心のうきはあふじに鳴ゝのことはたづめど篝ば見てもかきかぐはうき世を知るうちがらつか露のみめぬるおな
世を思つつかはなり立も草のつぎるじはとなわもなそし月もなりける
法ちにたるうちはらけりにも聞くかりは思やここそむ
見るうらがたに
めけてふる桜によき世の音に雁がめや引から物を見る
やがり
----- diversity: 1.2
----- Generating with seed: "ぞやも花のさかりにひとめ見し木のもとさへや秋はさびしき
桜こそ思しらすれ咲きにほ"
ぞやも花のさかりにひとめ見し木のもとさへや秋はさびしき
桜こそ思しらすれ咲きにほこと言はほを
かてしるらむことはながきも来て
もがやる風に心つそらましだてそふ
時何はれむ舟とと思ひやむるとや
身らびがらし
身そ世れども経の春かみぞ見る
優ぐりにいまさましきさぐさと過しがめよりても忘れねや
頼ましき中の柱にまるぬるかな
さんのみむ世にあふば見し人や
ならむ舟に吹ればる野をわかて行かぐきあや問な
をくれて海い世にかる舟を見むしながぐるなつべもがむかなし松によかつのお思水にもあれず
うらは色に恋しらてわかばふかなみにてす見しかげ潮や見ししものすめる物はいづにもかきつしてて見る
なくとぬん心しばかりける
よそ秋の

...

もろからにおきはとり花のあはぢに知らまさはむぐさかこのあはとは宿もながめに風のやはれに
ひけり世にはことありきみへやるるかゝこの世にかゝ見ぬは時は人もいきざりしき
絶え
----- diversity: 1.0
----- Generating with seed: "さはげる
人知れず神のゆるしを待ちしまにここらつれなき世を過ぐすかな
なべて世の"
さはげる
人知れず神のゆるしを待ちしまにここらつれなき世を過ぐすかな
なべて世のうには松にかけて山のは人にためしきたればはれをいづねよりはねば花ささりにも聞くなくらむかたになくしきあふの色やはりに里こそめる山よろもいくもふりてさすさはさにこともおきとらや頼まさる鳥のあやならぬ世にいとで尋世の知らんそも君をへだつぶ
あすれとぞ思ふ
いつてかえしき
あやときけり
おろしらにおらつらにいつもかは思ど残しすらん
つつるかな
いろもろそふき身には深きくと思
やむしきあひとやはり人におるるかな
のみやはにやよは言ひの空をとをめたるる袖
あすらとなびむきよろもいつけとを尋ねゆくとゞ言しきかよへても見したかりてぞ
見てこそ見し
おならぬみにぞかりけり
さゝの春のうらはぎぬ見ぬは深いつつもの別かし行をくふらざしにかかひやは思はほ鳥の旅まひなりしき
思ふりぬ心にうちはとれき空におの空をきがめに風の別れぬはと思ひしを
なくらん煙ことぞとせき
かざし世にかがふとけりずしよ人のたれかははらけて
----- diversity: 1.2
----- Generating with seed: "さはげる
人知れず神のゆるしを待ちしまにここらつれなき世を過ぐすかな
なべて世の"
さはげる
人知れず神のゆるしを待ちしまにここらつれなき世を過ぐすかな
なべて世のうにかよは色にこことな
亡えををばふぬもなそめぐ
さすみやしはいかにてあらぬ身にへてわがむかきを見し人や桜にいづはあはれぐさはいかなしき心も
あればつらなれぬ松に人にいまはげをやことなく
秋はへしかりにことらませず
瀬れは世のうきしを人かたにて行をくさらざりけん
さがにむとくほたる家に袖をきくしくらむす茅みだつかな
いともきくも知らのやかつにもひをきにしをむかし
あめぬぬに山の若がむ
亡わせしかげを見ぬもの花にた影はかりからを忍はかな
雲のさと
笛の波ちもろもいふせじしきをうちにて行きたるりしき

身をうたにし袖かな
つゝならばおぼほどぞに数ならぬを身にいつもかきかけてながめてんもさき心と人のまる
おるからに心をうらにけむ
よろもたつてとてて立ち寄る袖もあらぬをならむ
見てころも思ふまが身それしきゆかなりてこの世に願まる袖うらみぬるるかいかならぬ世のうらがめもかき水のすみはらけりにらむすもとにほかるらの袖をつけてはれてありやし
いかなるは時はいともなき立ちそぎる山よひのにとををくれぬとの露にかりせむ
この香をなびなき
いとがたきば
かけて世に知らまり
もろもたたぬとに行くゑらぬべき
思ひのじとを思ふうたがめし
それぬれの瀬にとはうらの春のかけみを人の色や恋はられをしなむすものたりにぞ消えぬ心なりせし
いかなくは身にうきもよそに見むとり
めぬるしふかなしほのあはなりぬも見るうつにものをむかしきてん瀬にいつましきをむくもひけるえし
人のつむく秋は立ち知
----- diversity: 0.5
----- Generating with seed: "野辺にまじるとも露のあだ名をわれにかけめや
花といへば名こそあだなれをみなへしな"
野辺にまじるとも露のあだ名をわれにかけめや
花といへば名こそあだなれをみなへしなかつれの花にかけりならぬ世にぞとりにしをまたる野思のうらに老どらぬまさん
いかつらの世にながかそうらけり
もろかけしを思ひとめる袖
あさだれの別る人は思やはる身な
おきてもいかにとを忘れぬると人もたがめるらむかたにならん
いつつかなき心ならでしをしなき心にて行をむぐふかに知らぬ若らむことにはかるあはれの色やいかがと知る川しな
あさだ見し程ぼえしげき心のどもをへてわかか知らずつらなりしき
かかりてて行く
かきつらけれ
かけふもなく音に投がよろほどをくるべき
思ひのましぞどもならしき
花の香をへだつらなる
山里の雪むくもあらぬはけふひとほどやひすらん
してことあらをおるかざしならぬ世に
あさかりはいつけてならぬ我身な
うき世をちにけり
あされぐひの空をみえぬはせむ
ひき世のうきにたちわが身にむめるらふるるかな
いつもかきかきに山のをはたぐかすて見し
よろにもあふかな
よろもならべき
よそぼれた
----- diversity: 1.0
----- Generating with seed: "野辺にまじるとも露のあだ名をわれにかけめや
花といへば名こそあだなれ

思は中の春にまどへぬ君に雪かりによ
世のうきに老どへ
----- diversity: 1.2
----- Generating with seed: "のはいまぞかなしき
ひとりゐて嘆きしよりは海人のすむかたをかくてぞ見るべかりける"
のはいまぞかなしき
ひとりゐて嘆きしよりは海人のすむかたをかくてぞ見るべかりけるりにいかなるらん
心つへどものがむししおめてぎがためむこるはかけてもろがねかゝかながらむ藤な
もいまたへぬも萩がみそめか年君が思ふ人に心もはべはあはれとそましやませ身とは思ひこそやる
かけれをして雪かかまるひやろはらざし水
暮かつつうつひあふ瀬きをみなへてだつかながほにつむくく
世につねども清ふる松よりのあ
さゝなむ人もあらぬまどうきによ
と夜を見き猶かざねかす思ひをながむるりの露にまさるける木のかしをしらむふる心ちはかけしき人の柱めだのはじづこりに心を結びぬかな
さみさめにいつけきを袖のむすがにもおしほじと垣へてもさらねね来にもとの垣がをばくもひこそ守かすべき
もなりしもおらぬ涙にとがてしやぐかふひよりだのなか
ゝおばかへるよ
あと見ても絶えしませき衣れたづすめけん
朝つもかすかける山の花に
吹てわか葉ぞ見し
はゝ間又にひしるまでそふるあまこともなかましをほどやはるる花にすすみの霜うへの
Epoch 52/60
7433/7433 [==============================] - 12s 2ms/step - loss: 0.0906

----- Generating text after Epoch: 51
----- diversity: 0.2
----- Generating with seed: "にひとり宿もる秋の夜の月
いつとかはおどろかすべき明けぬ夜の夢さめてとか言ひしひ"
にひとり宿もる秋の夜の月
いつとかはおどろかすべき明けぬ夜の夢さめてとか言ひしひとすらのうちに老をくくもあふかけてをたまし袖かき
なこしゐしせむおもろも涙をうたる涙かながめしほたる心にほくをくとたれの春と行く先るほのにける
きゝ音に音もなくらむことはとませきうらみわたる雁の月を
山里の秋をいづこともかわかきに山の露をたそれぬるせしことぞときもの山まの世をおらぬまにうつつけてしもの山の秋のたちびなりぬな
空きみ世をおらつけてなら衣しげきのとととま

あま舟ぞかきにうきみし人の思はぬはに
年の中の風ものうへみ見れ
山里の香をあだ見てて人も知らずそませてもかははぎぬる木のかく
くとつるより
あふればべといふるるはあさめけるあふも経そふるゝにことのはかかたへずはまづ雲のがにことはかやりほはれと見て
へだし世のあはいとゞも思ひとやる袖を思ひや
ことなくとなく音に投がめもしてて雪にかぎるらん
むすゑとらましとり
我涙を見てもつのかけしにまがふばば見む人にあまめけて誰かなき人の色に心におりせぬに秋のあはれはいづことにはかりしきてわがなみ身にいつつも露のかへりに
絶えぬまにうき世をあめぬ世のうらに知らず雲居るなく
あは舟ての世にかきなくと聞くる夜に心をくへるらしをしせしと聞くる袖うき
心はりにしをのうぐさ
----- diversity: 1.0
----- Generating with seed: "ぼゆる秋の空哉
また人になれける紬の移り香をわが身にしめてうらみつる哉
みなれぬ"
ぼゆる秋の空哉
また人になれける紬の移り香をわが身にしめてうらみつる哉
みなれぬしとのあた雲は立ち別れもきごともなき立のそれしのぼちにてほれぬまに立かるまどぞある間かな
立ちつるなく聞くはれける野の人にいかくま雲雲の山にひとめぎりける
しこのある深にみやまひゆくとな
うらめ刈し山のわがむかへて見る
花の香を心のとこに露を知らぬあるじを雨かな
山のこのよりばものはみぎる人はいかにともふきわがたにてて行たちばかすかへる心にともなりしき
うちに見し人をおてて松の山どもぎたるねなしならばましほのをしかかつまのにやかはまひぬべに霞のゆかこにはかに
けしきあらばども花の朝露にもある松な
それしやすらん
いとならし山のみちにぞふる鳴ちにわびて立ち真るる袖しき
を思ふどひ葉き瀬るみは絶ゆくほずにける
しほどたるよろにもぎてわがかひねめととと言ふば方さははあれれども猶かけしし
年ことのどどたのにけん
あらだなし浪
にひともあふかなき人にためぬまぬとにおどしけてあれのしやまさふ影みしまし
----- diversity: 1.2
----- Generating with seed: "ぼゆる秋の空哉
また人になれける紬の移り香をわが身にしめてうらみつる哉
みなれぬ"
ぼゆる秋の空哉
また人になれける紬の移り香をわが身にしめてうらみつる哉
みなれぬしなり声にみや

7433/7433 [==============================] - 12s 2ms/step - loss: 0.0760

----- Generating text after Epoch: 58
----- diversity: 0.2
----- Generating with seed: "この秋は逢坂山を霧なへだてそ
かげ広みたのみし松や枯れにけん下葉ちり行としの暮哉"
この秋は逢坂山を霧なへだてそ
かげ広みたのみし松や枯れにけん下葉ちり行としの暮哉
さここの世をみこそ見れ
とここそ荻をみやはむすや
ならさそのかしここそ宿れ
花の香のたにかけて見れどもひなるかかは思ひや
あふからをむすもころかな
かてりしかけふよりのしはとなりしも君がうへだつなになかつかすてて見る人に心にもぢる風に君ならむ風にながめそ浦づくはすらむ
初雁ををうつてあはれとも思ふも知らずわかるゝかけてならぬ世に
ありぬ雲のうへりけるかに身をうき世を
あだ見てもならまし
風吹のとも露のかきならぬを萩がにいかが空を思ひやませせむ
うきつれし別れしなりべき
うらめても思ふまでのふるまもなき亡の人の風にひとまでふ影もの山まためねめめならむうほたかしかく
ひきて人のすがよならぬ袖になくもいくなき人の色に心にとり見れを山のこともしきたれしるらのととも思ふもなれぬ世のうらなへの空ひとぞ見む
たちにわかる世のかぎは色はさはがたきほどもからまにおどらつけ木のかげはあはなれども聞くる身にお
----- diversity: 0.5
----- Generating with seed: "この秋は逢坂山を霧なへだてそ
かげ広みたのみし松や枯れにけん下葉ちり行としの暮哉"
この秋は逢坂山を霧なへだてそ
かげ広みたのみし松や枯れにけん下葉ちり行としの暮哉
さここのはとなみやれどもの衣たちけふはきけふよりばためにためるらひけるらな
いつてもなき立ち知らぬ見し人のゆかしきてずにたるまるる身を
あられぬ程くうらむ花のたち寄る浪
つつねどもあふけふのかやしき
つれても知らかりか行をみやうき世のうらのを
ななき身に生つてあはれと見ぬかな
つゝにもかくはじとけりけらやさほ鳥に花の名をばたましかげはまる身に
おのかたのころもをしどもかきにはことなれ
と人をいとぞふるうちもなき死やためぬまにおどらぬ見し
い